# Naive Bayes - Trabalho

## Questão 1

Implemente um classifacor Naive Bayes para o problema de predizer a qualidade de um carro. Para este fim, utilizaremos um conjunto de dados referente a qualidade de carros, disponível no [UCI](https://archive.ics.uci.edu/ml/datasets/car+evaluation). Este dataset de carros possui as seguintes features e classe:

** Attributos **
1. buying: vhigh, high, med, low
2. maint: vhigh, high, med, low
3. doors: 2, 3, 4, 5, more
4. persons: 2, 4, more
5. lug_boot: small, med, big
6. safety: low, med, high

** Classes **
1. unacc, acc, good, vgood


## Alunos
Bruno Dias

Eduardo Almeida

In [2]:
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt
from sklearn import preprocessing
from math import sqrt
import pandas as pd
import numpy as np
import random
import math

In [3]:
def load_csv(filename):
    dataset = cat_to_number(pd.read_csv(filename, header=None))
    return np.array(dataset).astype(np.float)

In [4]:
def cat_to_number(dataset):
    dic = {'vhigh':4, 'high':3, 'med':2, 'low':1, 'big':3,
           'small':1, 'unacc':1, 'acc':2, 'good':3, 'vgood':4,
           'more':5, '5more':6}
    dataset = dataset.replace(dic)
    return dataset

In [5]:
def calculateProbability(x, mean, stdev):
    
    if stdev == 0:
        return 1. if x == mean else .1
    
    exponent = math.exp(-(math.pow(x-mean,2)/(2*math.pow(stdev,2))))
    return (1 / (math.sqrt(2*math.pi) * math.pow(stdev, 2))) * exponent

In [6]:
def summarizeByClass(dataset):
    separated = separateByClass(dataset)
    summaries = {}
    for classValue, instances in separated.items():
        summaries[classValue] = summarize(instances)
    return summaries

In [7]:
def summarize(dataset):
    summaries = [(mean(attribute), stdev(attribute)) for attribute in zip(*dataset)]
    del summaries[-1]
    return summaries

In [8]:
def mean(numbers):
    return sum(numbers)/float(len(numbers))
 
def stdev(numbers):
    avg = mean(numbers)
    variance = sum([pow(x-avg,2) for x in numbers])/float(len(numbers)-1)
    return math.sqrt(variance)

In [9]:
def separateByClass(dataset):
    separated = {}
    for i in range(len(dataset)):
        vector = dataset[i]
        if (vector[-1] not in separated):
            separated[vector[-1]] = []
        separated[vector[-1]].append(vector)
    return separated

In [10]:
def predict(summaries, inputVector):
    probabilities = calculateClassProbabilities(summaries, inputVector)
    bestLabel, bestProb = None, -1
    for classValue, probability in probabilities.items():
        if bestLabel is None or probability > bestProb:
            bestProb = probability
            bestLabel = classValue
    return bestLabel

In [11]:
def calculateClassProbabilities(summaries, inputVector):
    probabilities = {}
    for classValue, classSummaries in summaries.items():
        probabilities[classValue] = 1
        for i in range(len(classSummaries)):
            mean, stdev = classSummaries[i]
            x = inputVector[i]
            probabilities[classValue] *= calculateProbability(x, mean, stdev)
    return probabilities

In [12]:
def getPredictions(summaries, testSet):
    predictions = []
    for i in range(len(testSet)):
        result = predict(summaries, testSet[i])
        predictions.append(result)
    return predictions

In [13]:
def getAccuracy(testSet, predictions):
    correct = 0
    for i in range(len(testSet)):
        if testSet[i][-1] == predictions[i]:
            correct += 1
    return (correct/float(len(testSet))) * 100.0

In [36]:
filename = 'carData.csv'
ds = load_csv(filename)
ds_train, ds_test = train_test_split(ds, test_size = 0.4)
pred = getPredictions(summarizeByClass(ds), ds_test)

In [37]:
NaiveBayesManual = getAccuracy(ds_test, pred)
NaiveBayesManual

71.09826589595376

## Questão 2
Crie uma versão de sua implementação usando as funções disponíveis na biblioteca SciKitLearn para o Naive Bayes ([veja aqui](http://scikit-learn.org/stable/modules/naive_bayes.html)) 

In [38]:
from sklearn.naive_bayes import GaussianNB

Y_train = ds_train[:,6]
Y_test = ds_test[:,6]

X_train = ds_train[:,0:6]
X_test = ds_test[:,0:6]

clf = GaussianNB()
clf.fit(X_train, Y_train)

GaussianNB(priors=None)

In [39]:
pred = clf.predict(X_test)

In [40]:
NaiveBayesSciKitLearn = getAccuracy(ds_test, pred)
NaiveBayesSciKitLearn

78.03468208092485

## Questão 3

Analise a acurácia dos dois algoritmos e discuta a sua solução.

In [41]:
NaiveBayesManual

71.09826589595376

In [42]:
NaiveBayesSciKitLearn

78.03468208092485